In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier


In [3]:
training_data = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

y = training_data.Transported
training_data.drop(['Transported', 'Name'], axis=1, inplace=True)
X = training_data


In [4]:
# Load the data
training_data = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

y = training_data.Transported
training_data.drop(['Transported'], axis=1, inplace=True)
X = training_data

# Get categorical and numerical cols 
cat_cols = [col for col in X.columns if X[col].dtype == 'object']
num_cols = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]

In [5]:
 param_grid = { 
    'model__n_estimators': [200,400,600,800,1000],
    'model__max_features': ['auto', 'sqrt', 'log2'],
    'model__max_depth' : [5,10,15,20,25,30,35,40,45,50],
    'model__criterion' :['gini', 'entropy']
}

# Create the preprocessing pipelines
num_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('scaler', StandardScaler())
])

cat_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('numerical', num_pipe, num_cols),
    ('categorical', cat_pipe, cat_cols)
])

# Define the model
model = RandomForestClassifier(n_estimators=1200, random_state=42)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

# Split the data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=.8, test_size=.2, random_state=42)

# Fit the model
grid_search_model = GridSearchCV(estimator=pipeline, param_grid=param_grid, n_jobs=4 )
grid_search_model.fit(X_train, y_train)

# Make predictions
pred = grid_search_model.predict(X_valid)

# Evaluate the Model
score = accuracy_score(y_valid, pred)

print("accuracy_score: ", score)


accuracy_score:  0.7498562392179413


In [6]:
pred = grid_search_model.predict(test_data)
final_output = pd.DataFrame({'PassengerId': test_data["PassengerId"], "Transported": pred})
final_output.to_csv('submission.csv',index=False)